In [ ]:



import pandas as pd
import numpy as np
from time import sleep
from memoise import memoise
from fPortfolio import *
from statsmodels.tsa.tsatools import lagmat


def model_performance(modelName, fromRound, toRound):
    username = modelName.lower()
    res = round_model_performances(username=username, tournament=8)
    res = (res.loc[(res['roundNumber'] >= fromRound) & (res['roundNumber'] <= toRound)]
           [['tc', 'tcPercentile', 'roundResolved', 'corrWMetamodel', 'corr', 'corrPercentile', 'roundNumber']])
    return res


if not 'mem_model_performance' in globals():
    mem_model_performance = memoise(model_performance)


def build_RAW(model_df, relative=False):
    model_names = model_df['ModelName']
    model_starts = model_df['Starting Era']
    model_ends = model_df['Last Era']
    RAW = pd.DataFrame()
    for i in range(len(model_names)):
        sleep(0.2)
        print(model_names[i])
        temp_corr = mem_model_performance(model_names[i], model_starts[i], model_ends[i])
        temp_corr = (temp_corr[['roundNumber', 'corrPercentile']] if relative else
                     temp_corr[['roundNumber', 'corr']])
        temp_corr['name'] = model_names[i] + '_corr'
        temp_corr.columns = ['roundNumber', 'score', 'name']
        RAW = RAW.append(temp_corr)
        temp_tc = mem_model_performance(model_names[i], model_starts[i], model_ends[i])
        temp_tc = (temp_tc[['roundNumber', 'tcPercentile']] if relative else
                     temp_tc[['roundNumber', 'tc']])
        temp_tc['name'] = model_names[i] + '_tc'
        temp_tc.columns = ['roundNumber', 'score', 'name']
        RAW = RAW.append(temp_tc)
    data_ts = (RAW.reset_index()
               .pivot_table(index='roundNumber', columns='name', values='score')
               .reset_index(drop=True))
    return data_ts


def tangency_portfolio(daily_data):
    TS = timeSeries(daily_data)
    spec = portfolioSpec()
    setType(spec) = "CVAR"
    setSolver(spec) = "solveRglpk.CVAR"
    optimized = tangencyPortfolio(TS, spec=spec, constraints="LongOnly")
    return optimized


def minvariance_portfolio(daily_data):
    TS = timeSeries(daily_data)
    spec = portfolioSpec()
    setType(spec) = "CVAR"
    setSolver(spec) = "solveRglpk.CVAR"
    optimized = minvariancePortfolio(TS, spec=spec, constraints="LongOnly")
    return optimized


def cleanup_portfolio(portfolio):
    weights = pd.DataFrame(getWeights(portfolio))
    weights.columns = ['weight']
    weights['name'] = weights.index
    corr_outcomes = weights[weights['name'].str.contains('_corr')]
    corr_outcomes.columns = ['name', 'corr_weight']
    corr_outcomes['name'] = corr_outcomes['name'].str.replace('(_corr|_tc)', '')
    tc_outcomes = weights[weights['name'].str.contains('_tc')]
    tc_outcomes.columns = ['name', 'tc_weight']
    tc_outcomes['name'] = tc_outcomes['name'].str.replace('(_corr|_tc)', '')
    outcome = pd.merge(corr_outcomes, tc_outcomes, how='outer')
    outcome['corr_weight'] = outcome['corr_weight'].fillna(0)
    outcome['tc_weight'] = outcome['tc_weight'].fillna(0)
    outcome = outcome.loc[outcome['corr_weight'] + outcome['tc_weight'] > 0.001]
    return outcome


def virtual_returns(merged_portfolio, data_ts, types='both'):
    if types == 'both':
        included = (merged_portfolio.loc[merged_portfolio['corr_weight'] > 0, 'name'] + '_corr'
                    + merged_portfolio.loc[merged_portfolio['tc_weight'] > 0, 'name'] + '_tc')
    else:
        included = merged_portfolio.loc[:, 'name'] + '_' + types
    daily_data = data_ts.loc[:, included]
    daily_data = daily_data.dropna()
    ewSpec = portfolioSpec()
    setType(ewSpec) = "CVAR"
    setSolver(ewSpec) = "solveRglpk.CVAR"
    weight_list = []
    for n in daily_data.columns:
        if n in merged_portfolio.loc[:, 'name'] + '_corr':
            weight_list.append(merged_portfolio.loc[merged_portfolio['name'] + '_corr' == n, 'corr_weight'].values[0])
        elif n in merged_portfolio.loc[:, 'name'] + '_tc':
            weight_list.append(merged_portfolio.loc[merged_portfolio['name'] + '_tc' == n, 'tc_weight'].values[0])
    setWeights(ewSpec) = weight_list
    port2 = feasiblePortfolio(timeSeries(daily_data), spec=ewSpec, constraints="LongOnly")
    return [np.round(getTargetReturn(port2)[0], 4),
            np.round(getTargetRisk(port2)[0], 4),
            np.round(getTargetRisk(port2)[2], 4),
            np.round(getTargetRisk(port2)[3], 4),
            samplesize= daily_data.shape[0]]